<a href="https://colab.research.google.com/github/guimalcantara/noticiaRSS/blob/main/Extra%C3%A7%C3%A3o_de_Not%C3%ADcias_via_RSS_com_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install feedparser beautifulsoup4


In [ ]:
!pip install selenium webdriver-manager beautifulsoup4 requests python-dateutil


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
from datetime import datetime
from pathlib import Path

import feedparser
import requests
from bs4 import BeautifulSoup
from dateutil import parser as date_parser

# -----------------------------------------------------------------------------
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
from datetime import datetime
from pathlib import Path

import feedparser
import requests
from bs4 import BeautifulSoup
from dateutil import parser as date_parser

# -----------------------------------------------------------------------------
# CONFIGURAÇÕES
# -----------------------------------------------------------------------------
FEED_URL    = "https://about.instagram.com/pt-br/blog/announcements"
OUTPUT_FILE = Path("avisos_blog.json")
START_DATE  = datetime(2025, 1, 1)
HEADERS     = {"User-Agent": "Mozilla/5.0"}

# -----------------------------------------------------------------------------
# 1) Parse do RSS
# -----------------------------------------------------------------------------
feed = feedparser.parse(FEED_URL)
if feed.bozo:
    raise RuntimeError(f"Erro ao ler feed: {feed.bozo_exception}")

# -----------------------------------------------------------------------------
# 2) Itera entradas e filtra por data
# -----------------------------------------------------------------------------
avisos = []
for entry in feed.entries:
    # published_parsed pode ser None em alguns feeds
    pub_date = None
    if hasattr(entry, "published"):
        pub_date = date_parser.parse(entry.published)
    elif hasattr(entry, "updated"):
        pub_date = date_parser.parse(entry.updated)
    if not pub_date or pub_date < START_DATE:
        continue

    url   = entry.link
    title = entry.title
    # resumo (HTML ou texto)
    summary = getattr(entry, "summary", "")

    # opcional: buscar conteúdo completo na página
    try:
        resp = requests.get(url, headers=HEADERS)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        body = soup.select_one(".wysiwyg, .ContentBody") or soup
        paragraphs = [p.get_text(strip=True) for p in body.find_all("p")]
        full_content = "\n\n".join(paragraphs)
    except Exception:
        full_content = summary

    avisos.append({
        "title":   title,
        "url":     url,
        "date":    pub_date.isoformat(),
        "summary": summary,
        "content": full_content
    })
    print(f"[OK] {title}")

# -----------------------------------------------------------------------------
# 3) Grava JSON
# -----------------------------------------------------------------------------
with OUTPUT_FILE.open("w", encoding="utf-8") as f:
    json.dump(avisos, f, ensure_ascii=False, indent=2)

print(f"\nConcluído: {len(avisos)} avisos salvos em {OUTPUT_FILE}")


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
from pathlib import Path
from datetime import datetime

import requests
from bs4 import BeautifulSoup
from dateutil import parser as date_parser

# -----------------------------------------------------------------------------
# CONFIGURAÇÃO
# -----------------------------------------------------------------------------
BASE_URL     = "https://about.instagram.com/pt-br/blog/announcements"
OUTPUT_PATH  = Path("avisos_announcements.json")
HEADERS      = {"User-Agent": "Mozilla/5.0"}
HASH_TAG     = "#AVISOS"
START_DATE   = datetime(2025, 1, 1)  # opcional

# -----------------------------------------------------------------------------
# 1) Busca cada página numerada de “Announcements”
# -----------------------------------------------------------------------------
def fetch_announcements_page(page: int) -> BeautifulSoup | None:
    url = BASE_URL if page == 1 else f"{BASE_URL}/page/{page}"
    resp = requests.get(url, headers=HEADERS)
    if resp.status_code != 200:
        return None
    soup = BeautifulSoup(resp.text, "html.parser")
    # busca cartão de anúncio
    cards = soup.select("article, .ContentItem")
    return soup if cards else None

# -----------------------------------------------------------------------------
# 2) Filtra cartões que contenham #AVISOS e extrai suas URLs
# -----------------------------------------------------------------------------
def extract_avisos_urls(soup: BeautifulSoup) -> list[str]:
    urls = []
    for card in soup.select("article, .ContentItem"):
        text = card.get_text(" ", strip=True).upper()
        if HASH_TAG not in text:
            continue
        link = card.find("a", href=True)
        if link:
            urls.append(link["href"].strip())
    # remove duplicatas mantendo ordem
    seen, unique = set(), []
    for u in urls:
        if u not in seen:
            unique.append(u); seen.add(u)
    return unique

# -----------------------------------------------------------------------------
# 3) Para cada URL, faz download e extrai título, data e corpo completo
# -----------------------------------------------------------------------------
def fetch_full_post(url: str) -> dict | None:
    resp = requests.get(url, headers=HEADERS)
    resp.raise_for_status()
    p = BeautifulSoup(resp.text, "html.parser")

    # título
    h = p.find(["h1", "h2"])
    title = h.get_text(strip=True) if h else ""

    # data
    t = p.find("time")
    date_iso = t.get("datetime", "") if t else ""
    # opcional: filtrar por data mínima
    if date_iso:
        try:
            dt = date_parser.parse(date_iso, dayfirst=True)
            if dt < START_DATE:
                return None
        except:
            pass

    # corpo
    body = p.select_one(".wysiwyg, .ContentBody") or p
    paras = [pt.get_text(strip=True) for pt in body.find_all("p")]
    content = "\n\n".join(paras)

    return {"url": url, "title": title, "date": date_iso, "content": content}

# -----------------------------------------------------------------------------
# 4) Fluxo principal
# -----------------------------------------------------------------------------
def main():
    page     = 1
    all_urls = []

    # percorre páginas até não encontrar novos cartões
    while True:
        soup = fetch_announcements_page(page)
        if soup is None:
            break
        urls = extract_avisos_urls(soup)
        if not urls:
            break
        all_urls.extend(urls)
        page += 1

    # extrai conteúdo completo
    avisos = []
    for u in all_urls:
        try:
            post = fetch_full_post(u)
            if post:
                avisos.append(post)
                print(f"[OK] {post['title']}")
        except Exception as e:
            print(f"[ERRO] {u} → {e}")

    # grava JSON
    with OUTPUT_PATH.open("w", encoding="utf-8") as f:
        json.dump(avisos, f, ensure_ascii=False, indent=2)

    print(f"\nConcluído: {len(avisos)} avisos salvos em {OUTPUT_PATH}")

if __name__ == "__main__":
    main()


In [ ]:
# Abaixo segue o código solicitado, com explicações detalhadas em português para uso em ambiente Jupyter/Colab

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm

def fetch_sitemap_urls(sitemap_url):
    """
    Recupera URLs do sitemap que apontem para a seção 'announcements' em português.
    Retorna lista de tuplas (url, lastmod).
    """
    resp = requests.get(sitemap_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.content, "xml")

    items = []
    for url in soup.find_all("url"):
        loc = url.loc.text
        if "/pt-br/blog/announcements/" in loc:
            lastmod = url.lastmod.text
            pub_dt = datetime.fromisoformat(lastmod)
            items.append((loc, pub_dt))
    return items

def fetch_full_content(url):
    """
    Faz requisição à página e extrai conteúdo principal via <article> ou div com class 'blog-post-content'.
    """
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.content, "html.parser")
        article = soup.find("article") or soup.find("div", class_="blog-post-content")
        return article.get_text(separator="\n").strip() if article else ""
    except:
        return ""

def main():
    # 1) URL do sitemap oficial de anúncios
    sitemap_url = "https://about.instagram.com/pt-br/announcements/sitemap.xml"

    # 2) Recupera URLs e datas
    entries = fetch_sitemap_urls(sitemap_url)

    data = []
    for url, pub_dt in tqdm(entries, desc="Baixando anúncios"):
        conteudo = fetch_full_content(url)

        # Extração segura do título
        page_soup = BeautifulSoup(requests.get(url).content, "html.parser")
        title_element = page_soup.find("h1")
        title = title_element.get_text(strip=True) if title_element else ""

        data.append({
            "titulo": title,
            "link": url,
            "data_publicacao": pub_dt,
            "conteudo_completo": conteudo  # aqui agora vai todo o texto
        })

    # 3) Criar DataFrame
    df = pd.DataFrame(data)
    return df

# Execução
df_announcements = main()
print(df_announcements.head())


In [ ]:
# Abaixo segue o código solicitado, com explicações detalhadas em português para uso em ambiente Jupyter/Colab

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm

def fetch_sitemap_urls(sitemap_url):
    """
    Recupera URLs do sitemap que apontem para a seção 'announcements' em português.
    Retorna lista de tuplas (url, lastmod).
    """
    resp = requests.get(sitemap_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.content, "xml")

    items = []
    for url in soup.find_all("url"):
        loc = url.loc.text
        if "/pt-br/blog/announcements/" in loc:
            lastmod = url.lastmod.text
            pub_dt = datetime.fromisoformat(lastmod)
            items.append((loc, pub_dt))
    return items

def fetch_all_text_elements(soup):
    """
    Extrai texto de todos os elementos relevantes da página para montar conteúdo completo.
    """
    sections = []

    # Título principal
    titulo = soup.find("h1")
    if titulo:
        sections.append(f"# {titulo.get_text(strip=True)}")

    # Subtítulos e seções
    for tag in soup.find_all(["h2", "h3", "h4", "p", "ul", "ol"]):
        text = tag.get_text(separator=" ", strip=True)
        if text:
            sections.append(text)

    return "\n\n".join(sections).strip()

def fetch_full_page_content(url):
    """
    Recupera o HTML da página e extrai o título e todo o conteúdo útil.
    """
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.content, "html.parser")

        titulo_tag = soup.find("h1")
        titulo = titulo_tag.get_text(strip=True) if titulo_tag else "(sem título)"

        conteudo_completo = fetch_all_text_elements(soup)

        return titulo, conteudo_completo
    except:
        return "(erro ao obter título)", ""

def main():
    sitemap_url = "https://about.instagram.com/pt-br/announcements/sitemap.xml"
    entries = fetch_sitemap_urls(sitemap_url)

    data = []
    for url, pub_dt in tqdm(entries, desc="Baixando e extraindo dados"):
        titulo, conteudo = fetch_full_page_content(url)
        data.append({
            "titulo": titulo,
            "link": url,
            "data_publicacao": pub_dt,
            "conteudo_completo": conteudo
        })

    df = pd.DataFrame(data)
    return df

# Execução
df_anuncios_completos = main()
df_anuncios_completos.head()


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, date
import time
from tqdm.notebook import tqdm # Use tqdm.notebook para ambiente Jupyter/Colab

# --- Configurações ---
BASE_URL = "https://about.instagram.com"
ANNOUNCEMENTS_PATH = "/pt-br/blog/announcements"
FULL_LISTING_URL = BASE_URL + ANNOUNCEMENTS_PATH

# Definir a data de início para filtrar (início do ano atual)
START_DATE = date(datetime.now().year, 1, 1)

# Headers para simular um navegador (pode ajudar a evitar bloqueios simples)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- Funções Auxiliares ---

def parse_date(date_str):
    """Tenta parsear a string de data 'Month Day, Year' para um objeto date."""
    try:
        # Ex: "February 22, 2024"
        # O local 'pt_BR' pode ser necessário dependendo do sistema, mas o formato %B (nome completo do mês)
        # geralmente funciona com nomes de mês em inglês por padrão em muitos ambientes Python.
        # Se der erro de localidade, pode precisar configurar.
        # import locale
        # locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8') # Pode variar dependendo do OS
        return datetime.strptime(date_str, "%B %d, %Y").date()
    except ValueError:
        print(f"Erro ao parsear data: {date_str}")
        return None

def get_page_content(url):
    """Faz a requisição GET para uma URL e retorna o objeto BeautifulSoup."""
    try:
        response = requests.get(url, headers=HEADERS, timeout=10) # Timeout de 10 segundos
        response.raise_for_status() # Lança exceção para status de erro (4xx ou 5xx)
        return BeautifulSoup(response.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar a página {url}: {e}")
        return None

def extract_article_links_and_dates(soup):
    """Extrai URLs, títulos e datas da página de listagem."""
    articles_list = []
    # Seletor baseado na inspeção manual da página (PODE MUDAR!)
    # Container principal dos artigos
    articles_container = soup.find('div', class_='_aajr')

    if not articles_container:
        print("Não foi possível encontrar o container principal dos artigos. Verifique os seletores.")
        return []

    # Itens individuais de artigo
    article_items = articles_container.find_all('div', class_='_aajs')

    if not article_items:
        print("Não foi possível encontrar itens de artigos. Verifique os seletores.")
        return []

    for item in article_items:
        link_tag = item.find('a', href=True)
        date_tag = item.find('div', class_='_aajt') # Seletor para a data (PODE MUDAR!)

        if link_tag and date_tag:
            relative_url = link_tag['href']
            full_url = BASE_URL + relative_url
            title = link_tag.get_text(strip=True)
            date_str = date_tag.get_text(strip=True)
            article_date = parse_date(date_str)

            if article_date:
                articles_list.append({
                    'title': title,
                    'url': full_url,
                    'date': article_date
                })
        else:
             print(f"Aviso: Item de artigo encontrado sem link ou data esperados: {item.prettify()[:200]}...")


    return articles_list

def extract_article_content(soup):
    """Extrai o texto completo do corpo de um artigo individual."""
    # Seletor para o container principal do conteúdo (PODE MUDAR!)
    content_container = soup.find('div', class_='_aajk _aajp') # Exemplo de seletores combinados

    if not content_container:
        # Tentar encontrar o título caso o container principal falhe
        title_tag = soup.find('h1') or soup.find('h2')
        title_text = title_tag.get_text(strip=True) if title_tag else "Título não encontrado"
        print(f"Não foi possível encontrar o container de conteúdo para o artigo '{title_text}'. Verifique os seletores.")
        # Retorna texto do corpo inteiro da página (menos ideal) ou uma mensagem de erro
        # return soup.get_text(strip=True) # Ou retornar o texto bruto da página inteira
        return "Conteúdo principal não encontrado com os seletores definidos."

    # Extrair texto de todos os parágrafos dentro do container
    paragraphs = content_container.find_all('p')
    if paragraphs:
        # Juntar o texto dos parágrafos, adicionando quebras de linha
        full_text = "\n\n".join([p.get_text(strip=True) for p in paragraphs])
    else:
        # Se não encontrar parágrafos, tenta pegar todo o texto do container principal
        full_text = content_container.get_text(strip=True)
        if not full_text:
             full_text = "Nenhum texto de parágrafo ou conteúdo encontrado no container."


    return full_text

# --- Processo Principal ---

print(f"Buscando a lista de artigos em: {FULL_LISTING_URL}")
listing_soup = get_page_content(FULL_LISTING_URL)

if listing_soup:
    all_articles_info = extract_article_links_and_dates(listing_soup)

    # Filtrar artigos pela data (desde o início do ano)
    articles_this_year = [
        art for art in all_articles_info if art['date'] >= START_DATE
    ]

    print(f"Encontrados {len(all_articles_info)} artigos listados na página.")
    print(f"Filtrando para artigos desde {START_DATE.strftime('%d/%m/%Y')}.")
    print(f"Serão processados {len(articles_this_year)} artigos que atendem ao filtro de data.")

    extracted_data = []

    # Usar tqdm para mostrar o progresso ao processar cada artigo
    print("\nIniciando extração do conteúdo completo dos artigos...")
    for article_info in tqdm(articles_this_year, desc="Processando artigos"):
        article_url = article_info['url']
        article_title = article_info['title']
        article_date = article_info['date']

        article_soup = get_page_content(article_url)

        if article_soup:
            full_content = extract_article_content(article_soup)

            extracted_data.append({
                'Título': article_title,
                'URL': article_url,
                'Data': article_date,
                'Conteúdo Completo': full_content
            })
        else:
            extracted_data.append({
                 'Título': article_title,
                 'URL': article_url,
                 'Data': article_date,
                 'Conteúdo Completo': "Erro ao carregar ou parsear a página do artigo."
            })

        # Pequeno delay para ser educado
        time.sleep(0.5) # Atraso de meio segundo entre as requisições

    # Criar o DataFrame pandas
    df = pd.DataFrame(extracted_data)

    # Opcional: Ordenar por data
    df = df.sort_values(by='Data', ascending=False).reset_index(why='drop')

    print("\nExtração concluída.")
    print("DataFrame criado:")
    print(df.head())
    print(f"\nTotal de artigos extraídos: {len(df)}")

else:
    print("Não foi possível buscar a página de listagem. Encerrando o script.")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, date
import time
from tqdm.notebook import tqdm # Use tqdm.notebook para ambiente Jupyter/Colab

# --- Configurações ---
BASE_URL = "https://about.instagram.com"
ANNOUNCEMENTS_PATH = "/pt-br/blog/announcements"
FULL_LISTING_URL = BASE_URL + ANNOUNCEMENTS_PATH

# Definir a data de início para filtrar (início do ano atual)
START_DATE = date(datetime.now().year, 1, 1)

# Headers para simular um navegador (pode ajudar a evitar bloqueios simples)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Configurações para retentativa (retry)
MAX_RETRIES = 5
RETRY_DELAY_SECONDS = 5 # Tempo de espera inicial para retentativa 429

# --- Funções Auxiliares ---

def parse_date(date_str):
    """Tenta parsear a string de data 'Month Day, Year' para um objeto date."""
    try:
        # Ex: "February 22, 2024"
        return datetime.strptime(date_str, "%B %d, %Y").date()
    except ValueError:
        print(f"Erro ao parsear data: {date_str}")
        return None

def get_page_content(url, retries=0):
    """
    Faz a requisição GET para uma URL, retorna o objeto BeautifulSoup.
    Inclui retentativa para erro 429.
    """
    try:
        print(f"Buscando: {url}") # Indica qual URL está sendo buscada
        response = requests.get(url, headers=HEADERS, timeout=15) # Aumentar timeout
        response.raise_for_status() # Lança exceção para status de erro (4xx ou 5xx) exceto 429 tratado abaixo

        return BeautifulSoup(response.content, 'html.parser')

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 429:
            if retries < MAX_RETRIES:
                wait_time = RETRY_DELAY_SECONDS * (2 ** retries) # Espera exponencial
                print(f"Erro 429 (Too Many Requests) para {url}. Tentando novamente em {wait_time} segundos (Tentativa {retries + 1}/{MAX_RETRIES})...")
                time.sleep(wait_time)
                return get_page_content(url, retries + 1) # Chama a função recursivamente
            else:
                print(f"Erro 429 persistente para {url} após {MAX_RETRIES} tentativas. Pulando...")
                return None
        else:
            print(f"Erro HTTP {e.response.status_code} ao buscar a página {url}: {e}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar a página {url}: {e}")
        return None


def extract_article_links_and_dates(soup):
    """Extrai URLs, títulos e datas da página de listagem."""
    articles_list = []
    # Seletor baseado na inspeção manual da página (PODE MUDAR!)
    # Container principal dos artigos
    articles_container = soup.find('div', class_='_aajr')

    if not articles_container:
        print("Não foi possível encontrar o container principal dos artigos. Verifique os seletores (_aajr).")
        return []

    # Itens individuais de artigo
    article_items = articles_container.find_all('div', class_='_aajs')

    if not article_items:
        print("Não foi possível encontrar itens de artigos. Verifique os seletores (_aajs).")
        return []

    for item in article_items:
        link_tag = item.find('a', href=True)
        date_tag = item.find('div', class_='_aajt') # Seletor para a data (PODE MUDAR!)

        if link_tag and date_tag:
            relative_url = link_tag['href']
            full_url = BASE_URL + relative_url
            title = link_tag.get_text(strip=True)
            date_str = date_tag.get_text(strip=True)
            article_date = parse_date(date_str)

            if article_date:
                articles_list.append({
                    'title': title,
                    'url': full_url,
                    'date': article_date
                })
            else:
                 print(f"Aviso: Data não parseada para o item: {date_str}. Pulando item.")
        else:
             print(f"Aviso: Item de artigo encontrado sem link ou data esperados. Pulando item.")


    return articles_list

def extract_article_content(soup):
    """Extrai o texto completo do corpo de um artigo individual."""
    # Seletor para o container principal do conteúdo (PODE MUDAR!)
    # Pode haver múltiplos containers dependendo do layout da página.
    # Vamos tentar encontrar o container com a classe específica,
    # mas se falhar, talvez pegar o texto de todos os parágrafos dentro de uma área mais geral.
    content_container = soup.find('div', class_='_aajk _aajp') # Exemplo de seletores combinados

    if not content_container:
        # Tentar encontrar o título para o print de erro
        title_tag = soup.find('h1') or soup.find('h2')
        title_text = title_tag.get_text(strip=True) if title_tag else "Título não encontrado"
        print(f"Aviso: Não foi possível encontrar o container de conteúdo principal para o artigo '{title_text}'. Verifique os seletores (_aajk _aajp). Tentando extrair parágrafos de uma área mais ampla.")

        # Tentar encontrar parágrafos em uma área potencialmente mais estável
        # Pode ser necessário ajustar este seletor também
        paragraphs = soup.select('div._a8z9 p, div._aajk p') # Tenta parágrafos dentro de _a8z9 ou _aajk
        if paragraphs:
             full_text = "\n\n".join([p.get_text(strip=True) for p in paragraphs])
             if full_text:
                 return full_text
             else:
                 return "Conteúdo principal não encontrado com os seletores definidos, mesmo em áreas amplas."
        else:
             return "Nenhum parágrafo de conteúdo encontrado com os seletores definidos."


    # Extrair texto de todos os parágrafos dentro do container específico
    paragraphs = content_container.find_all('p')
    if paragraphs:
        # Juntar o texto dos parágrafos, adicionando quebras de linha
        full_text = "\n\n".join([p.get_text(strip=True) for p in paragraphs])
    else:
        # Se não encontrar parágrafos no container específico, tenta pegar todo o texto dele
        full_text = content_container.get_text(strip=True)
        if not full_text:
             full_text = "Nenhum texto de parágrafo ou conteúdo encontrado no container específico."

    return full_text


# --- Processo Principal ---

print(f"Iniciando busca pela lista de artigos em: {FULL_LISTING_URL}")

# Adiciona um pequeno delay antes da primeira requisição também
time.sleep(2) # Espera 2 segundos antes de começar

listing_soup = get_page_content(FULL_LISTING_URL)

if listing_soup:
    all_articles_info = extract_article_links_and_dates(listing_soup)

    # Filtrar artigos pela data (desde o início do ano)
    articles_this_year = [
        art for art in all_articles_info if art['date'] >= START_DATE
    ]

    print(f"Encontrados {len(all_articles_info)} artigos listados na página (podem ser mais do que o filtro de data).")
    print(f"Filtrando para artigos desde {START_DATE.strftime('%d/%m/%Y')}.")
    print(f"Serão processados {len(articles_this_year)} artigos que atendem ao filtro de data.")

    extracted_data = []

    # Usar tqdm para mostrar o progresso ao processar cada artigo
    print("\nIniciando extração do conteúdo completo dos artigos...")
    # Adiciona um pequeno delay antes de começar a iterar sobre os artigos também
    time.sleep(1)

    for article_info in tqdm(articles_this_year, desc="Processando artigos"):
        article_url = article_info['url']
        article_title = article_info['title']
        article_date = article_info['date']

        article_soup = get_page_content(article_url) # get_page_content já tem retry/delay para 429

        if article_soup:
            full_content = extract_article_content(article_soup)

            extracted_data.append({
                'Título': article_title,
                'URL': article_url,
                'Data': article_date,
                'Conteúdo Completo': full_content
            })
        else:
            extracted_data.append({
                 'Título': article_title,
                 'URL': article_url,
                 'Data': article_date,
                 'Conteúdo Completo': "Erro ao carregar ou parsear a página do artigo individual após retentativas."
            })

        # Pequeno delay para ser educado (além da retentativa no get_page_content)
        # Isso garante um tempo mínimo entre as requisições de artigos diferentes.
        time.sleep(1) # Atraso de 1 segundo entre processar cada artigo

    # Criar o DataFrame pandas
    df = pd.DataFrame(extracted_data)

    # Opcional: Ordenar por data
    df = df.sort_values(by='Data', ascending=False).reset_index(drop=True)

    print("\nExtração concluída.")
    print("DataFrame criado:")
    print(df.head())
    print(f"\nTotal de artigos extraídos: {len(df)}")

else:
    print("Não foi possível buscar a página de listagem após retentativas. Encerrando o script.")

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL da página de anúncios do blog do Instagram
url = 'https://about.instagram.com/pt-br/blog/announcements'

# Faz a requisição HTTP
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Lista para armazenar resultados
resultados = []

# A estrutura real da página pode conter blocos de artigos em divs específicas
# Aqui buscamos todos os links de artigos com base em observações práticas
for artigo in soup.find_all('a', href=True):
    href = artigo['href']
    texto = artigo.get_text(strip=True)

    # Filtro: considerar apenas links que levam para posts e têm título visível
    if texto and href.startswith('/pt-br/blog/') and 'announcements' not in href:
        resultados.append((texto, f"https://about.instagram.com{href}"))

# Salva em arquivo de texto
with open('conteudo_instagram.txt', 'w', encoding='utf-8') as arquivo:
    for titulo, link in resultados:
        arquivo.write(f"Título: {titulo}\n")
        arquivo.write(f"Link: {link}\n\n")
